<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment04/Assignment_4_Gibbs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING DATASETS AND INSTALLING REQUIRED DEPENDENCIES

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: kodikarthik21 (use `wandb login --relogin` to force relogin)


True

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy import exp
from math import log
numpy_rng = np.random.RandomState(1234)

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Fashion_MNIST/fashion-mnist_train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Fashion_MNIST/fashion-mnist_test.csv')

# DATASET PREPROCESSING

In [ ]:
train_data_df = train_df.drop('label', axis=1) #dropping the labels column to separate the data from the labels
train_data_np = train_data_df.to_numpy() #data matrix
train_bin = np.where(train_data_np > 127, 1, 0) #converting the data matrix to binary
train_data = train_bin.reshape(train_bin.shape[0], 784) #reshaping
train_labels = train_df['label'].to_frame().to_numpy() #labels
train_data,val_data,train_labels,val_labels = train_test_split(train_data, train_labels, test_size=0.1, random_state=42)

In [ ]:
test_data_df = test_df.drop('label', axis=1)
test_data_np = test_data_df.to_numpy()
test_bin = np.where(test_data_np > 127, 1, 0)
test_data = test_bin.reshape(test_bin.shape[0], 784)
test_labels = test_df['label'].to_frame().to_numpy()

# SIMPLE LOGISTIC REGRESSION MODEL

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
def train_test(x_train, y_train, x_test,y_test):
  logisticRegr = LogisticRegression(max_iter = 1000,multi_class='multinomial') #multi-class logistic regression model for 1000 iterations
  logisticRegr.fit(x_train, y_train)
  y_pred = logisticRegr.predict(x_test)
  pred = logisticRegr.predict_proba(x_test)
  loss = log_loss(y_test,pred) #cross-entropy loss
  accuracy = accuracy_score(y_test, y_pred) #accuracy
  return loss, accuracy

# HELPER FUNCTIONS FOR RBM AND RBM MODEL

In [ ]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))

In [ ]:
class RBM(object):
     def __init__(self, input=None, n_visible=2, n_hidden=3, W=None, hbias=None, vbias=None):
        
        if W is None:
            initial_W = np.random.randn(n_visible, n_hidden) * 0.01
            W = initial_W

        if hbias is None:
            hbias = np.random.randn(n_hidden) * 0.01  # initialize h bias 0

        if vbias is None:
            vbias = np.random.randn(n_visible) * 0.01 # initialize v bias 0

        self.n_visible = n_visible 
        self.n_hidden = n_hidden   
        self.input = input
        self.W = W
        self.hbias = hbias
        self.vbias = vbias

def gibbs_sampling(rbm, lr=0.1, k=200, r=10):

  h_sig_initial, samp_h_initial = prob_hv(rbm,np.random.randn(rbm.input.shape[0], rbm.input.shape[1]))

  chain_start = samp_h_initial

  for step in range(k):
      print("Sampling k={}".format(step))
      if step == 0:
          sig_v, samp_v = prob_vh(rbm,samp_h_initial)
      else:
          sig_v, samp_v = prob_vh(rbm,samp_h)
      sig_h, samp_h = prob_hv(rbm,samp_v)

  dW_2 = numpy.zeros((rbm.n_visible,rbm.n_hidden))
  dvbias_2 = numpy.zeros(rbm.n_visible)
  dhbias_2 = numpy.zeros(rbm.n_hidden)

  for step in range(r):
    print("Sampling r={}".format(step))
    nv_means, nv_samples,\
    nh_means, nh_samples = gibbs_hvh(rbm,nh_samples)
    dW_2 += numpy.dot(nv_samples.T, nh_means)
    dvbias_2 += numpy.sum(nv_samples, axis=0)
    dhbias_2 += numpy.sum(nh_means, axis=0)

  rbm.W += lr * (numpy.dot(rbm.input.T, ph_mean)
                  - dW_2)
  rbm.vbias += lr * numpy.mean(rbm.input - dvbias_2, axis=0)
  rbm.hbias += lr * numpy.mean(ph_sample - dhbias_2, axis=0)

#-----------------SAMPLING(H/V)-------------------------
def prob_hv(rbm, v):
    sig_h = sigmoid(np.dot(v, rbm.W) + rbm.c)
    samp_h = numpy_rng.binomial(size=sig_h.shape, n=1, p=sig_h)
    return [sig_h, samp_h]

#-----------------SAMPLING(V/H)-------------------------
def prob_vh(rbm, h):
    sig_v =  sigmoid(np.dot(h, rbm.W.T) + rbm.b)
    samp_v = numpy_rng.binomial(size=sig_v.shape, n=1, p=sig_v) 
    return [sig_v, samp_v]

#--------------- COMPUTE HIDDEN REPRESENTATION OF VALIDATION AND TEST DATA------
def get_hidden_reps(rbm, v,t):
    h_v = sigmoid(np.dot(v, rbm.W) + rbm.c)
    h_t = sigmoid(np.dot(t, rbm.W) + rbm.c)
    return h_v,h_t      

#--TRAIN THE RBM AND RETURN HIDDEN REPRESENTATIONS OF VALIDATION AND TEST DATA--
def train_rbm(rbm, v,t,learning_rate=0.05, k = 200, r=10):
    gibbs_sampling(rbm,lr=learning_rate,k=k,r=r)
    return get_hidden_reps(rbm,v,t)

# SWEEP

In [ ]:
def swp():
#-------------DEFAULT HYPERPARAMETERS-------------------------------------------
  hyperparameter_defaults = dict(
    n_hidden=64,
    k = 200,
    r = 10
  )
#---------------SETTING UP THE SWEEP--------------------------------------------
  wandb.init(project="Assignment-4", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "{}_h_layers_{}_k_{}_r- Gibbs".format(config.n_hidden, config.k,config.r)
  n_visible=784
  rbm = RBM(input=train_data, n_visible=n_visible, n_hidden=config.n_hidden)

#----------------RUNNING THE MODEL----------------------------------------------
  for epoch in range(5):

    hidden_rep,hidden_rep_test= train_rbm(rbm,val_data,test_data,k = config.k,r=config.r)
    print('Training epoch: %d' %epoch)
    loss, accuracy = train_test(hidden_rep, val_labels, hidden_rep_test,test_labels)
    print('Loss:{} Accuracy:{}'.format(loss,accuracy))
    metrics = {'epoch':epoch, 'accuracy': accuracy, 'loss': loss}
    wandb.log(metrics)
  wandb.run.finish()

In [ ]:
sweep_config = {
  "name": "Sweep - Gibbs",
  "method": "grid",
  "project": "Assignment-4",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "n_hidden": {
            "values":[64,128,256]
        },
        "k": {
            "values":[200,300]
        },
        "r": {
            "values":[10,20,30]
        },
    }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: nna1wkpl
Sweep URL: https://wandb.ai/kodikarthik21/uncategorized/sweeps/nna1wkpl


In [ ]:
#wandb.agent('4ehftw3l', swp)